In [1]:
# from podcast_ad_skipper.Spectrogram_Conversion import create_spectrogram
import pandas as pd
from podcast_ad_skipper.google_cloud import *
import librosa
from podcast_ad_skipper.params import *
from podcast_ad_skipper.data_preparation import *

from google.auth.exceptions import GoogleAuthError
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery


In [2]:
storage_client = auth_gc()

Authenticated successfully! ✅


In [3]:
prefixes =GCP_PREFIXES[0]
bucket_name = BUCKET_NAME

In [4]:
file_list = retrieve_files_in_folder(storage_client, bucket_name, prefixes)

In [5]:
open_file = open_gcs_file(file_list[0])

In [6]:
features = get_features_model(file_list)

Processing files: total 5837


In [7]:
features[0][0].shape

(96, 64, 3)

In [8]:
import pandas as pd
from podcast_ad_skipper.main import transform_features_into_dataframe

In [23]:
data = transform_features_into_dataframe(features)
data

,spectrogram,labels,seconds,durations,podcast_names
0,"[[[0.3966400921344757, 0.3966400921344757, 0.3...",0,1000,5864,billionairepersonalitydisorder
1,"[[[0.15291857719421387, 0.15291857719421387, 0...",0,1001,5864,billionairepersonalitydisorder
2,"[[[0.1725386381149292, 0.1725386381149292, 0.1...",0,1002,5864,billionairepersonalitydisorder
3,"[[[0.5269443392753601, 0.5269443392753601, 0.5...",0,1003,5864,billionairepersonalitydisorder


In [24]:
type(data["spectrogram"][0])

list

In [11]:
data['seconds'] = data['seconds'].astype(int)
data['durations'] = data['durations'].astype(int)
data["labels"] = data['labels'].astype(int)

In [52]:
data#['spectrogram'][0]

,spectrogram,labels,seconds,durations,podcast_names
0,"[[[0.3966400921344757, 0.3966400921344757, 0.3...",0,1000,5864,billionairepersonalitydisorder
1,"[[[0.15291857719421387, 0.15291857719421387, 0...",0,1001,5864,billionairepersonalitydisorder
2,"[[[0.1725386381149292, 0.1725386381149292, 0.1...",0,1002,5864,billionairepersonalitydisorder
3,"[[[0.5269443392753601, 0.5269443392753601, 0.5...",0,1003,5864,billionairepersonalitydisorder


In [27]:
data.shape

(4, 5)

In [37]:
data_to_upload = [
    {
        'spectrogram': data["spectrogram"][i],  # Array of floats
        'labels': data["labels"][i]     # Corresponding integer
    }
    for i in range(data.shape[0])
]

In [43]:
data_to_upload[3]["labels"]

0

In [46]:
table_id = "podcast-ad-skipper.Numpy_Arrays_Dataset.test4"
bq_client = auth_gc_bigquery()
    # Create df out of the np arrays, to upload to bq
    # Use WRITE_APPEND to add to the existing table
job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        schema=[
            bigquery.SchemaField("spectrogram",  "FLOAT", mode="REPEATED"),
            bigquery.SchemaField("labels", "INTEGER")
            # bigquery.SchemaField("seconds", "INTEGER"),
            # bigquery.SchemaField("durations", "INTEGER"),
            # bigquery.SchemaField("podcast_names", "STRING")
        ]
    )
job = bq_client.load_table_from_json(data_to_upload, table_id, job_config=job_config)
print(job.result())
print(f"Appended rows to {table_id}")

Authenticated successfully with BigQuery! ✅


BadRequest: 400 Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: JSON processing encountered too many errors, giving up. Rows: 1; errors: 1; max bad: 0; error percent: 0; reason: invalid, message: Error while reading data, error message: JSON parsing error in row starting at position 0: Nested arrays not allowed.

In [14]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"

In [13]:
client = bq_client
query = """
SELECT spectrogram from podcast-ad-skipper.Numpy_Arrays_Dataset.test3
"""

    # Run the query
query_job = client.query(query)  # Make an API request.

    # Wait for the job to complete and get the results
results = query_job.result()  # Waits for the job to complete.

results


In [14]:
for row in results:
    test = row[0]

In [15]:
np.array(test)

array('[[[0.52694434 0.52694434 0.52694434]\n  [0.20000769 0.20000769 0.20000769]\n  [0.29453546 0.29453546 0.29453546]\n  ...\n  [0.1793058  0.1793058  0.1793058 ]\n  [0.17196724 0.17196724 0.17196724]\n  [0.18345404 0.18345404 0.18345404]]\n\n [[0.5398639  0.5398639  0.5398639 ]\n  [0.23499829 0.23499829 0.23499829]\n  [0.39060026 0.39060026 0.39060026]\n  ...\n  [0.29102594 0.29102594 0.29102594]\n  [0.3723137  0.3723137  0.3723137 ]\n  [0.25340876 0.25340876 0.25340876]]\n\n [[0.57968205 0.57968205 0.57968205]\n  [0.393878   0.393878   0.393878  ]\n  [0.47804725 0.47804725 0.47804725]\n  ...\n  [0.5705231  0.5705231  0.5705231 ]\n  [0.57804185 0.57804185 0.57804185]\n  [0.32729322 0.32729322 0.32729322]]\n\n ...\n\n [[0.12095577 0.12095577 0.12095577]\n  [0.3135188  0.3135188  0.3135188 ]\n  [0.3265841  0.3265841  0.3265841 ]\n  ...\n  [0.08064917 0.08064917 0.08064917]\n  [0.28398946 0.28398946 0.28398946]\n  [0.3297113  0.3297113  0.3297113 ]]\n\n [[0.1271886  0.1271886  0.127188

In [22]:
test.replace(" ", ",")

'[[[0.52694434,0.52694434,0.52694434]\n,,[0.20000769,0.20000769,0.20000769]\n,,[0.29453546,0.29453546,0.29453546]\n,,...\n,,[0.1793058,,0.1793058,,0.1793058,]\n,,[0.17196724,0.17196724,0.17196724]\n,,[0.18345404,0.18345404,0.18345404]]\n\n,[[0.5398639,,0.5398639,,0.5398639,]\n,,[0.23499829,0.23499829,0.23499829]\n,,[0.39060026,0.39060026,0.39060026]\n,,...\n,,[0.29102594,0.29102594,0.29102594]\n,,[0.3723137,,0.3723137,,0.3723137,]\n,,[0.25340876,0.25340876,0.25340876]]\n\n,[[0.57968205,0.57968205,0.57968205]\n,,[0.393878,,,0.393878,,,0.393878,,]\n,,[0.47804725,0.47804725,0.47804725]\n,,...\n,,[0.5705231,,0.5705231,,0.5705231,]\n,,[0.57804185,0.57804185,0.57804185]\n,,[0.32729322,0.32729322,0.32729322]]\n\n,...\n\n,[[0.12095577,0.12095577,0.12095577]\n,,[0.3135188,,0.3135188,,0.3135188,]\n,,[0.3265841,,0.3265841,,0.3265841,]\n,,...\n,,[0.08064917,0.08064917,0.08064917]\n,,[0.28398946,0.28398946,0.28398946]\n,,[0.3297113,,0.3297113,,0.3297113,]]\n\n,[[0.1271886,,0.1271886,,0.1271886,]\n,